In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd # 2.x to support pyarrow
import pyarrow as pa
import openpyxl # for reading xlsx with structure
import numpy as np
import plotly.express as px
import math
import re
from datetime import timedelta, datetime
import os

import tensorflow as tf
from tensorflow import keras

In [4]:
x_structure = pd.read_parquet('../data/pipeline/x_structure.parquet')
y_structure = pd.read_parquet('../data/pipeline/y_structure.parquet')

In [5]:
x_test = pd.read_parquet('../data/source/X_test.parquet')

In [6]:
submission1_ref = pd.read_excel('../data/source/sample_submission/submission_1.xlsx', index_col=0)
submission2_ref = pd.read_parquet('../data/source/sample_submission/sample_submission_2.parquet').set_index('DT')
submission3_ref = pd.read_parquet('../data/source/sample_submission/sample_submission_3.parquet')

In [22]:
MAX_TTE = 31 * 24 * 60 * 60

In [ ]:
submission2 = pd.DataFrame(index=x_test.index, columns=submission2_ref.columns)
submission3 = pd.DataFrame(index=x_test.index, columns=submission3_ref.columns)

for place in y_structure.index:
    try:
        submission2_slice = pd.read_parquet(f'../dist/submission2/{place}.parquet')
        submission2[submission2_slice.columns] = submission2_slice[submission2_slice.columns]
        submission3_slice = pd.read_parquet(f'../dist/submission3/{place}.parquet')
        submission3[submission3_slice.columns] = submission3_slice[submission3_slice.columns]
        print(f'Найден файл для {place}')
    except:
        print(f'Не найден файл для {place}')

submission2 = submission2.fillna(0)
submission3 = submission3.fillna(MAX_TTE)

In [147]:
# submission2

In [148]:
# submission3

In [0]:
submission1 = submission1_ref.copy()
submission1['machine'] = np.NaN
submission1['tm'] = np.NaN

labels = y_structure.unstack()
for (idx, row) in submission1_ref.iterrows():
    start = row['start']
    finish = row['finish']
    submission3_slice = submission3.loc[start:finish]
    smallest_mean_row = submission3_slice.mean().sort_values().reset_index().iloc[0]
    smallest_mean_value = smallest_mean_row[0]
    if smallest_mean_value < 0.8 * MAX_TTE:
        label = smallest_mean_row['index']
        label_row = labels[labels == label]
        machine, place = list(label_row.index)[0]
        submission1['machine'].at[idx] = machine
        submission1['tm'].at[idx] = label # @NOTE: Согласно примеру, ожидают в полной форме

In [149]:
# submission1
# submission1[submission1[['machine', 'tm']].notna().any(axis=1)]

In [150]:
# @NOET: Пример sub 1 был сохранен в экселе, но мы умеем только csv
submission1.to_csv('../dist/submission_1.csv')

# @NOTE: Пример sub 2 сохранен без индекса, хз насколько это будет важно при автоматической проверк
submission2.reset_index().to_parquet('../dist/submission_2.parquet')

submission3.to_parquet('../dist/submission_3.parquet')